In [1]:
import os
import pandas as pd
pd.options.display.max_columns = None
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from utils import searchVideosByListOfIds, fillCategoryIds

# region = 'GB'

In [ ]:
def searchVideo(query):
    api_key = os.environ.get('YT_API')

    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    youtube = googleapiclient.discovery.build(
        "youtube", "v3", developerKey=api_key)

    request = youtube.search().list(
        part="snippet",
        maxResults=3,
        q=query
    )
    response = request.execute()
    return response

### Uzupełnianie brakujących linków do thumbnaili

In [ ]:
import json
import pandas as pd

def fixThumbnailURL(row):
    response = searchVideo(row.title)
    thumbnailURL = response['items'][0]['snippet']['thumbnails']['default']['url']
    searchedVideoID = response['items'][0]['id']['videoId']
    
    if (searchedVideoID == row.video_id):
        row.thumbnail_link = thumbnailURL
    else:
        row.thumbnail_link = "N/A"
    return row

df = pd.read_csv(f"./../../our_data/Etap2/lilatee/{region}-06_sentiment.csv")
df_org = pd.read_csv(f"./../../our_data/Etap1/{region}.csv")
df_org = df_org[["video_id", "title"]]
df = pd.merge(df, df_org, on="video_id")

# CHYBA DZIAŁA, ALE USTAWIONE NA JEDEN WIERSZ PONIŻEJ
df[:1][df.mean_r.isna()].apply(lambda row: fixThumbnailURL(row), axis=1)

### Zabawa z pobieraniem filmów nietrending z tych samych kanałów, nieskończone

In [ ]:
df = pd.read_csv(f"./../../our_data/Etap1/{region}.csv", dtype={'publish_time': 'str'}, parse_dates=['publish_time'])
df["trending_date"] = df["trending_date"].apply(lambda x: pd.to_datetime(x, format="%y.%d.%m"))
df[:3]

In [ ]:
print(f'Liczba filmów: {df.shape[0]}')
print(f'Liczba unikalnych kanałów: {df["channel_title"].nunique()}')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Turn off x axis labels
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
# plt.title("")
sns.ecdfplot(data=df, x='channel_title')

In [ ]:
threshold = 0.35

In [ ]:
import numpy as np
n_of_videos = df.shape[0]
channel_nOfOccurences = df["channel_title"].value_counts().reset_index().values
channel_nOfOccurences

In [ ]:
channel_nOfOccurencesCumSum = channel_nOfOccurences.copy()
channel_nOfOccurencesCumSum[:, 1] = np.cumsum(channel_nOfOccurencesCumSum[:, 1])
channel_nOfOccurencesCumSum

In [ ]:
selected_id = None
for i, channel in enumerate(channel_nOfOccurencesCumSum):
    if channel[1]/n_of_videos > threshold:
        selected_id = i
        break
        
print(f"Liczba wybranych kanałów: {selected_id+1}")
chosen_channels = channel_nOfOccurences[:, 0][:selected_id+1]

In [ ]:
def searchChannelIdByName(channel_name):
    api_key = os.environ.get('YT_API')

    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    youtube = googleapiclient.discovery.build(
        "youtube", "v3", developerKey=api_key)

    request = youtube.channels().list(
        part="id",
        forUsername=channel_name
    )
    response = request.execute()
    return response['items'][0]['id']

# channelId = searchChannelIdByName("EminemVEVO")

In [ ]:
from datetime import datetime, timedelta

def getVideosOfChannelBetweenDates(channel_name, start_datetime, end_datetime):
    api_key = os.environ.get('YT_API')

    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    youtube = googleapiclient.discovery.build(
        "youtube", "v3", developerKey=api_key)

    request = youtube.search().list(
        part="snippet",
        channelId = searchChannelIdByName(channelName),
        type='video',
        publishedBefore=(last_trend_datetime - timedelta(days=30)).isoformat('T') + 'Z',
        publishedAfter=(first_trend_datetime + timedelta(days=30)).isoformat('T') + 'Z',
#         pageToken="CDIQAA",
        maxResults=500

    )
    
    return request.execute()

In [ ]:
channelName = 'EminemVEVO'
first_trend_datetime = min(df["trending_date"])
last_trend_datetime = max(df["trending_date"])

response = getVideosOfChannelBetweenDates(channelName, first_trend_datetime, last_trend_datetime )

In [ ]:
channel_nOfOccurences[:, 0]

In [ ]:
from pathlib import Path

last_channel_name_filepath = Path("./last_channel.txt")
last_channel_name = ''

if last_channel_name_filepath.is_file():
    with open(last_channel_filepath, 'r') as file:
        last_channel_name = file.read()
else:
    last_channel_name_filepath.mkdir(parents=True, exit_ok=True)

for channelName in channel_nOfOccurences[:, 0]:
    getVideosOfChannelBetweenDates(channelName, first_trend_datetime, last_trend_datetime)

### Pobieranie ground truth dla kategorii

In [3]:
import json
with open("./../../data/youtube_data/GB_category_id.json") as file:
    data = json.loads(file.read())
    
GB_cats = {}
for item in data['items']:
    GB_cats[item['id']] = item['snippet']['title']
GB_cats

{'1': 'Film & Animation',
 '2': 'Autos & Vehicles',
 '10': 'Music',
 '15': 'Pets & Animals',
 '17': 'Sports',
 '18': 'Short Movies',
 '19': 'Travel & Events',
 '20': 'Gaming',
 '21': 'Videoblogging',
 '22': 'People & Blogs',
 '23': 'Comedy',
 '24': 'Entertainment',
 '25': 'News & Politics',
 '26': 'Howto & Style',
 '27': 'Education',
 '28': 'Science & Technology',
 '30': 'Movies',
 '31': 'Anime/Animation',
 '32': 'Action/Adventure',
 '33': 'Classics',
 '34': 'Comedy',
 '35': 'Documentary',
 '36': 'Drama',
 '37': 'Family',
 '38': 'Foreign',
 '39': 'Horror',
 '40': 'Sci-Fi/Fantasy',
 '41': 'Thriller',
 '42': 'Shorts',
 '43': 'Shows',
 '44': 'Trailers'}

In [4]:
with open("./../../data/youtube_data/US_category_id.json") as file:
    data = json.loads(file.read())
    
US_cats = {}
for item in data['items']:
    US_cats[item['id']] = item['snippet']['title']
US_cats

{'1': 'Film & Animation',
 '2': 'Autos & Vehicles',
 '10': 'Music',
 '15': 'Pets & Animals',
 '17': 'Sports',
 '18': 'Short Movies',
 '19': 'Travel & Events',
 '20': 'Gaming',
 '21': 'Videoblogging',
 '22': 'People & Blogs',
 '23': 'Comedy',
 '24': 'Entertainment',
 '25': 'News & Politics',
 '26': 'Howto & Style',
 '27': 'Education',
 '28': 'Science & Technology',
 '29': 'Nonprofits & Activism',
 '30': 'Movies',
 '31': 'Anime/Animation',
 '32': 'Action/Adventure',
 '33': 'Classics',
 '34': 'Comedy',
 '35': 'Documentary',
 '36': 'Drama',
 '37': 'Family',
 '38': 'Foreign',
 '39': 'Horror',
 '40': 'Sci-Fi/Fantasy',
 '41': 'Thriller',
 '42': 'Shorts',
 '43': 'Shows',
 '44': 'Trailers'}

In [5]:
print(f"GB - US = {set(GB_cats).difference(set(US_cats))}")
print(f"US - GB = {set(US_cats).difference(set(GB_cats))}")

GB - US = set()
US - GB = {'29'}


### GB

In [9]:
df = pd.read_csv(f"./../../our_data/Etap2/GB-Etap2.csv")
df = df[~df.mean_r.isna()]
df["category_id"] = ""
df[:3]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,region,pub_date,pub_day_of_the_week,pub_year,pub_month,pub_day,pub_hour,tren_date,tren_day_of_the_week,tren_year,tren_month,tren_day,time_to_trend_in_days,dislikes/likes,likes/views,dislikes/views,comment_count/views,increase_views,increase_likes,increase_dislikes,increase_comms,times_in_trend,avg_views,avg_likes,avg_dislikes,avg_comms,avg_views_increase_per_hour,n_of_tags,title_clean,title_length,title_n_of_words,title_capital_letters,title_capital_letters_percent,title_small_letters,title_small_letters_percent,title_punctuation,title_punctuation_percent,title_n_of_emojis,desc_clean,desc_length,desc_n_of_words,desc_capital_letters,desc_capital_letters_percent,desc_small_letters,desc_small_letters_percent,desc_punctuation,desc_punctuation_percent,desc_list_of_urls,desc_n_of_urls,desc_n_of_emojis,isFacebook,isTwitter,isInstagram,region.1,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY,ocr_sentiment_compound,features_vec_resnet18
2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,,2017-11-10T17:00:03.000Z,"[""Eminem"", ""Walk"", ""On"", ""Water"", ""Aftermath/S...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,GB,2017-11-10 17:00:03,4.0,2017.0,11.0,10.0,17.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,4.291620,0.055142,0.045891,0.002531,0.007336,3380838.0,53222.0,4295.0,-1646.0,14.0,3.017875e+07,950319.000000,56545.571429,130640.5,3.998159e+06,6,eminem walk water audio ft beyoncé,42,8,6,0.142857,25,0.595238,4,0.095238,0,eminems new track walk water ft beyoncé availa...,594,34,29,0.048822,433,0.728956,9,0.158249,"[""http://shady.sr/WOWEminem"", ""https://goo.gl/...",14,0,True,True,True,GB,66.502830,71.743704,79.699635,83.992492,83.883918,83.748135,"[43570, 1539, 1228, 1489, 1782, 1839, 1918, 20...","[43570, 1539, 1228, 1489, 1782, 1839, 1918, 20...","[43570, 1539, 1228, 1489, 1782, 1839, 1918, 20...",6.470819,\r\n,NaN,wigy water an vevo,wigy water vevo,"[""[[25, 118], [173, 194]]"", ""[[282, 116], [468...","[177.0, 202.33333333333334]",177.0,202.333333,0.0,"[0.24609655141830444, 2.066065788269043, 4.700..."
3,PUTEiSjKwJU,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,,2017-11-13T02:30:38.000Z,"[""Salford City FC"", ""Salford City"", ""Salford"",...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...,GB,2017-11-13 02:30:38,0.0,2017.0,11.0,13.0,2.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,1.895382,0.062176,0.006934,0.000431,0.001329,49625.0,164.0,26.0,32.0,14.0,9.007371e+04,398.214286,40.857143,53.5,1.468464e+04,9,goal salford city v class 92 friend peninsula ...,76,14,8,0.105263,52,0.684211,1,0.013158,0,salford drew 44 class 92 friend newly opened p...,396,37,28,0.070707,283,0.714646,7,0.113636,"[""https://www.facebook.com/SalfordCityFC/"", ""h...",5,0,True,True,True,GB,86.534334,73.284271,69.150666,78.330501,71.675730,68.933206,"[38491, 1841, 1874, 1096, 801, 658, 612, 594, ...","[38491, 1841, 1874, 1096, 801, 658, 612, 594, ...","[38491, 1841, 1874, 1096, 801, 658, 612, 594, ...",6.839139,\r\n\r\ney tderr ott\r\n\r\nSTE a CLASS...,class,3 10 3 92 salford city class of 92 4 dudley2n ...,salford city class dudley2n dieseruvwe webbers...,"[""[[127, 260], [173, 277]]"", ""[[174, 262], [19...","[219.6, 284.9]",219.6,284.900000,0.0,"[0.08890726417303085, 0.28475505113601685, 0.4..."
5,AumaWl0TNBo,17.14.11,How My Relationship Started!,PointlessBlogVlogs,,2017-11-11T17:00:00.000Z,"[""pointlessblog"", ""pointlessblogtv"", ""pointles...",1182775,52708,1431,233

In [19]:
df = fillCategoryIds(df, 50) # First we try to minimize using units in YT API.
df[df["category_id"] == ''] = fillCategoryIds(df[df["category_id"] == ''], 1) # Next we fill missing values.

<ipython-input-18-ef394157e78b>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i:i+step]["category_id"] = cats_list
<ipython-input-18-ef394157e78b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i:i+step]["category_id"] = ''


In [20]:
df[df["category_id"] == '']

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,region,pub_date,pub_day_of_the_week,pub_year,pub_month,pub_day,pub_hour,tren_date,tren_day_of_the_week,tren_year,tren_month,tren_day,time_to_trend_in_days,dislikes/likes,likes/views,dislikes/views,comment_count/views,increase_views,increase_likes,increase_dislikes,increase_comms,times_in_trend,avg_views,avg_likes,avg_dislikes,avg_comms,avg_views_increase_per_hour,n_of_tags,title_clean,title_length,title_n_of_words,title_capital_letters,title_capital_letters_percent,title_small_letters,title_small_letters_percent,title_punctuation,title_punctuation_percent,title_n_of_emojis,desc_clean,desc_length,desc_n_of_words,desc_capital_letters,desc_capital_letters_percent,desc_small_letters,desc_small_letters_percent,desc_punctuation,desc_punctuation_percent,desc_list_of_urls,desc_n_of_urls,desc_n_of_emojis,isFacebook,isTwitter,isInstagram,region.1,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY,ocr_sentiment_compound,features_vec_resnet18
388,PCVtE3wRNPg,17.23.11,"Let's talk... YOUTUBE-MONEY, ZOELLA & PEOPLE C...",Caspar,,2017-11-20T17:08:03.000Z,"[""caspar lee"", ""talk"", ""youtube"", ""zoella"", ""y...",816123,78829,1815,3692,https://i.ytimg.com/vi/PCVtE3wRNPg/default.jpg,False,False,False,I just wanted to talk about a few things on my...,GB,2017-11-20 17:08:03,0.0,2017.0,11.0,20.0,17.0,2017-11-23 23:59:59,3.0,2017.0,11.0,23.0,3.286065,0.023025,0.096590,0.002224,0.004524,59390.0,3023.0,336.0,151.0,12.0,1.034877e+06,88880.833333,3216.916667,4247.583333,248358.765269,11,let talk youtubemoney zoella people changing,53,7,33,0.622642,7,0.132075,7,0.132075,0,wanted talk thing mind moment n follow caspar ...,413,34,27,0.065375,292,0.707022,15,0.147700,"[""http://instagram.com/caspar"", ""http://twitte...",7,0,False,True,True,GB,141.471042,101.775891,96.967980,112.352044,110.953154,111.977977,"[36978, 2643, 1281, 348, 714, 457, 197, 139, 1...","[36978, 2643, 1281, 348, 714, 457, 197, 139, 1...","[36978, 2643, 1281, 348, 714, 457, 197, 139, 1...",4.997187,\r\n,NaN,talk lets,talk let,"[""[[344, 130], [448, 168]]"", ""[[225, 133], [33...","[338.5, 149.75]",338.5,149.75,0.0,"[0.17629368603229523, 1.7783278226852417, 1.37..."
1814,oEtEBOy0ys8,18.02.02,THINGS WE ALL DID IN BRITISH SECONDARY SCHOOL,Luke Birch,,2018-01-25T20:41:17.000Z,"[""British Secondary School"", ""Things we all di...",108685,9646,105,2013,https://i.ytimg.com/vi/oEtEBOy0ys8/default.jpg,False,False,False,A few relatable things that we all do in Briti...,GB,2018-01-25 20:41:17,3.0,2018.0,1.0,25.0,20.0,2018-02-02 23:59:59,4.0,2018.0,2.0,2.0,8.137986,0.010885,0.088752,0.000966,0.018521,9595.0,687.0,14.0,165.0,22.0,2.612937e+05,19651.363636,369.136364,4106.681818,13355.269782,14,thing british secondary school,45,8,38,0.844444,0,0.000000,0,0.000000,0,relatable thing british secondary schoolbritis...,1012,80,167,0.165020,526,0.519763,133,0.209486,"[""https://twitter.com/LukeBirch"", ""http://inst...",10,0,True,True,True,GB,99.504589,94.060093,94.125723,87.152131,76.271129,76.082382,"[38292, 2178, 974, 331, 812, 386, 400, 196, 19...","[38292, 2178, 974, 331, 812, 386, 400, 196, 19...","[38292, 2178, 974, 331, 812, 386, 400, 196, 19...",6.700530,\r\n,NaN,seatches,seatches,"[""[[202, 184], [260, 225]]""]","[231.0, 204.5]",231.0,204.50,0.0,"[0.43880921602249146, 2.079106569290161, 2.361..."


In [26]:
df[~(df["category_id"] == '')].to_csv("./../../our_data/Etap3/GB-Etap3.csv", index=False)

### US

In [4]:
df = pd.read_csv(f"./../../our_data/Etap2/US-Etap2.csv")
df = df[~df.mean_r.isna()]
df["category_id"] = ""
df[:3]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,region,pub_date,pub_day_of_the_week,pub_year,pub_month,pub_day,pub_hour,tren_date,tren_day_of_the_week,tren_year,tren_month,tren_day,time_to_trend_in_days,dislikes/likes,likes/views,dislikes/views,comment_count/views,increase_views,increase_likes,increase_dislikes,increase_comms,times_in_trend,avg_views,avg_likes,avg_dislikes,avg_comms,avg_views_increase_per_hour,n_of_tags,title_clean,title_length,title_n_of_words,title_capital_letters,title_capital_letters_percent,title_small_letters,title_small_letters_percent,title_punctuation,title_punctuation_percent,title_n_of_emojis,desc_clean,desc_length,desc_n_of_words,desc_capital_letters,desc_capital_letters_percent,desc_small_letters,desc_small_letters_percent,desc_punctuation,desc_punctuation_percent,desc_list_of_urls,desc_n_of_urls,desc_n_of_emojis,isFacebook,isTwitter,isInstagram,region.1,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY,ocr_sentiment_compound,features_vec_resnet18
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,,2017-11-13T17:13:01.000Z,"[""SHANtell martin""]",748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,US,2017-11-13 17:13:01,0.0,2017.0,11.0,13.0,17.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,1.282616,0.051558,0.076869,0.003963,0.021318,1440216.0,30572.0,4184.0,8271.0,7.0,2.174423e+06,87968.428571,7001.714286,21342.285714,5.834748e+05,1,want talk marriage,34,7,28,0.823529,0,0.000000,0,0.000000,0,shantells channel httpswwwyoutubecomshantellma...,1379,159,248,0.179840,797,0.577955,36,0.110950,"[""https://www.youtube.com/shantellmartin"", ""ht...",23,0,True,True,True,US,83.791198,77.128183,74.980914,78.653796,73.570512,73.087883,"[38336, 2426, 1188, 690, 848, 602, 597, 711, 1...","[38336, 2426, 1188, 690, 848, 602, 597, 711, 1...","[38336, 2426, 1188, 690, 848, 602, 597, 711, 1...",6.818234,\r\n,NaN,NaN,NaN,[],"[-1, -1]",-1.00,-1.0,0.0000,"[0.028933335095643997, 1.5294386148452759, 0.4..."
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,,2017-11-13T07:30:00.000Z,"[""last week tonight trump presidency"", ""last w...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",US,2017-11-13 07:30:00,0.0,2017.0,11.0,13.0,7.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,1.687488,0.063240,0.040179,0.002541,0.005252,2213233.0,34599.0,3746.0,5581.0,7.0,5.027629e+06,136365.571429,10166.285714,18159.000000,1.433363e+06,4,trump presidency last week tonight john oliver...,62,10,11,0.177419,39,0.629032,3,0.048387,0,one year presidential election john oliver dis...,615,85,34,0.055285,466,0.757724,15,0.050407,"[""http://Facebook.com/LastWeekTonight"", ""http:...",3,0,True,True,False,US,53.131123,50.240133,55.462459,66.342402,62.884021,62.213755,"[48274, 4068, 2082, 1566, 1196, 1059, 1116, 10...","[48274, 4068, 2082, 1566, 1196, 1059, 1116, 10...","[48274, 4068, 2082, 1566, 1196, 1059, 1116, 10...",6.185228,\r\n,NaN,the trump presidency,trump presidency,"[""[[68, 169], [105, 178]]"", ""[[108, 169], [176...","[114.25, 173.5]",114.25,173.5,0.0000,"[0.34424084424972534, 2.8481504917144775, 0.59..."
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,,2017-11-13T11:00:04.000Z,"[""rhett and link"", ""gmm"", ""good mythical morni...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,US,2017-11-13 11:00:04,0.0,2017.0,11.0,13.0,11.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,1.541609,0.065474,0.029641,0.0

In [5]:
df = fillCategoryIds(df, 50) # First we try to minimize using units in YT API.
df[df["category_id"] == ''] = fillCategoryIds(df[df["category_id"] == ''], 1) # Next we fill missing values.

/mnt/d/Documents/Uczelnia/Magisterka_Semestr3/PED/PED_YT/notebooks/Etap3/utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i:i+step]["category_id"] = cats_list
/mnt/d/Documents/Uczelnia/Magisterka_Semestr3/PED/PED_YT/notebooks/Etap3/utils.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i:i+step, :]["category_id"] = ''


In [6]:
df[df["category_id"] == '']

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,region,pub_date,pub_day_of_the_week,pub_year,pub_month,pub_day,pub_hour,tren_date,tren_day_of_the_week,tren_year,tren_month,tren_day,time_to_trend_in_days,dislikes/likes,likes/views,dislikes/views,comment_count/views,increase_views,increase_likes,increase_dislikes,increase_comms,times_in_trend,avg_views,avg_likes,avg_dislikes,avg_comms,avg_views_increase_per_hour,n_of_tags,title_clean,title_length,title_n_of_words,title_capital_letters,title_capital_letters_percent,title_small_letters,title_small_letters_percent,title_punctuation,title_punctuation_percent,title_n_of_emojis,desc_clean,desc_length,desc_n_of_words,desc_capital_letters,desc_capital_letters_percent,desc_small_letters,desc_small_letters_percent,desc_punctuation,desc_punctuation_percent,desc_list_of_urls,desc_n_of_urls,desc_n_of_emojis,isFacebook,isTwitter,isInstagram,region.1,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY,ocr_sentiment_compound,features_vec_resnet18
2800,P2I7hQHOqNI,18.18.01,Martin Luther King - A Time to Break Silence,FacelesswithEyesOpen,,2011-03-27T01:52:47.000Z,"[""Martin Luther King"", ""MLK"", ""A Time to Break...",2896,30,1,3,https://i.ytimg.com/vi/P2I7hQHOqNI/default.jpg,False,False,False,Martin Luther King - A Time to Break Silence S...,US,2011-03-27 01:52:47,6.0,2011.0,3.0,27.0,1.0,2018-01-18 23:59:59,3.0,2018.0,1.0,18.0,2489.921667,0.033333,0.010359,0.000345,0.001036,48.0,2.0,0.0,0.0,2.0,2920.0,31.0,1.0,3.0,1.163089,3,martin luther king time break silence,44,9,7,0.159091,28,0.636364,1,0.022727,0,martin luther king time break silence starting...,3072,550,139,0.045247,2270,0.738932,67,0.02181,[],0,0,False,False,False,US,103.12169,103.12169,103.12169,78.496168,78.496168,78.496168,"[16701, 951, 951, 656, 811, 507, 4199, 639, 20...","[16701, 951, 951, 656, 811, 507, 4199, 639, 20...","[16701, 951, 951, 656, 811, 507, 4199, 639, 20...",6.881028,\r\n,NaN,NaN,NaN,[],"[-1, -1]",-1.0,-1.0,0.0,"[1.2514019012451172, 1.0734546184539795, 0.566..."


In [7]:
df[~(df["category_id"] == '')].to_csv("./../../our_data/Etap3/US-Etap3.csv", index=False)

### GB_US

In [8]:
df = pd.read_csv(f"./../../our_data/Etap2/GB_US-Etap2.csv")
df = df[~df.mean_r.isna()]
df["category_id"] = ""
df[:3]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,region,pub_date,pub_day_of_the_week,pub_year,pub_month,pub_day,pub_hour,tren_date,tren_day_of_the_week,tren_year,tren_month,tren_day,time_to_trend_in_days,dislikes/likes,likes/views,dislikes/views,comment_count/views,increase_views,increase_likes,increase_dislikes,increase_comms,times_in_trend,avg_views,avg_likes,avg_dislikes,avg_comms,avg_views_increase_per_hour,n_of_tags,title_clean,title_length,title_n_of_words,title_capital_letters,title_capital_letters_percent,title_small_letters,title_small_letters_percent,title_punctuation,title_punctuation_percent,title_n_of_emojis,desc_clean,desc_length,desc_n_of_words,desc_capital_letters,desc_capital_letters_percent,desc_small_letters,desc_small_letters_percent,desc_punctuation,desc_punctuation_percent,desc_list_of_urls,desc_n_of_urls,desc_n_of_emojis,isFacebook,isTwitter,isInstagram,region.1,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY,ocr_sentiment_compound,features_vec_resnet18
2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,,2017-11-10T17:00:03.000Z,"[""Eminem"", ""Walk"", ""On"", ""Water"", ""Aftermath/S...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,GB,2017-11-10 17:00:03,4.0,2017.0,11.0,10.0,17.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,4.291620,0.055142,0.045891,0.002531,0.007336,3380838.0,53222.0,4295.0,-1646.0,19.0,2.810155e+07,926578.736842,54665.315789,129223.0,3.998159e+06,6,eminem walk water audio ft beyoncé,42,8,6,0.142857,25,0.595238,4,0.095238,0,eminems new track walk water ft beyoncé availa...,594,34,29,0.048822,433,0.728956,9,0.158249,"[""http://shady.sr/WOWEminem"", ""https://goo.gl/...",14,0,True,True,True,GB,66.502830,71.743704,79.699635,83.992492,83.883918,83.748135,"[43570, 1539, 1228, 1489, 1782, 1839, 1918, 20...","[43570, 1539, 1228, 1489, 1782, 1839, 1918, 20...","[43570, 1539, 1228, 1489, 1782, 1839, 1918, 20...",6.470819,\n,NaN,wigy water an vevo,wigy water vevo,"[""[[25, 118], [173, 194]]"", ""[[282, 116], [468...","[177.0, 202.33333333333334]",177.0,202.333333,0.0,"[0.24609655141830444, 2.066065788269043, 4.700..."
3,PUTEiSjKwJU,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,,2017-11-13T02:30:38.000Z,"[""Salford City FC"", ""Salford City"", ""Salford"",...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...,GB,2017-11-13 02:30:38,0.0,2017.0,11.0,13.0,2.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,1.895382,0.062176,0.006934,0.000431,0.001329,49625.0,164.0,26.0,32.0,14.0,9.007371e+04,398.214286,40.857143,53.5,1.468464e+04,9,goal salford city v class 92 friend peninsula ...,76,14,8,0.105263,52,0.684211,1,0.013158,0,salford drew 44 class 92 friend newly opened p...,396,37,28,0.070707,283,0.714646,7,0.113636,"[""https://www.facebook.com/SalfordCityFC/"", ""h...",5,0,True,True,True,GB,86.534334,73.284271,69.150666,78.330501,71.675730,68.933206,"[38491, 1841, 1874, 1096, 801, 658, 612, 594, ...","[38491, 1841, 1874, 1096, 801, 658, 612, 594, ...","[38491, 1841, 1874, 1096, 801, 658, 612, 594, ...",6.839139,\n\ney tderr ott\n\nSTE a CLASS OF 92\n...,class,3 10 3 92 salford city class of 92 4 dudley2n ...,salford city class dudley2n dieseruvwe webbers...,"[""[[127, 260], [173, 277]]"", ""[[174, 262], [19...","[219.6, 284.9]",219.6,284.900000,0.0,"[0.08890726417303085, 0.28475505113601685, 0.4..."
5,AumaWl0TNBo,17.14.11,How My Relationship Started!,PointlessBlogVlogs,,2017-11-11T17:00:00.000Z,"[""pointlessblog"", ""pointlessblogtv"", ""pointles...",1182775,52708,1431,2333,

In [9]:
df = fillCategoryIds(df, 50) # First we try to minimize using units in YT API.
df[df["category_id"] == ''] = fillCategoryIds(df[df["category_id"] == ''], 1) # Next we fill missing values.

/mnt/d/Documents/Uczelnia/Magisterka_Semestr3/PED/PED_YT/notebooks/Etap3/utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i:i+step]["category_id"] = cats_list
/mnt/d/Documents/Uczelnia/Magisterka_Semestr3/PED/PED_YT/notebooks/Etap3/utils.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i:i+step, :]["category_id"] = ''


In [10]:
df[df["category_id"] == '']

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,region,pub_date,pub_day_of_the_week,pub_year,pub_month,pub_day,pub_hour,tren_date,tren_day_of_the_week,tren_year,tren_month,tren_day,time_to_trend_in_days,dislikes/likes,likes/views,dislikes/views,comment_count/views,increase_views,increase_likes,increase_dislikes,increase_comms,times_in_trend,avg_views,avg_likes,avg_dislikes,avg_comms,avg_views_increase_per_hour,n_of_tags,title_clean,title_length,title_n_of_words,title_capital_letters,title_capital_letters_percent,title_small_letters,title_small_letters_percent,title_punctuation,title_punctuation_percent,title_n_of_emojis,desc_clean,desc_length,desc_n_of_words,desc_capital_letters,desc_capital_letters_percent,desc_small_letters,desc_small_letters_percent,desc_punctuation,desc_punctuation_percent,desc_list_of_urls,desc_n_of_urls,desc_n_of_emojis,isFacebook,isTwitter,isInstagram,region.1,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY,ocr_sentiment_compound,features_vec_resnet18
388,PCVtE3wRNPg,17.23.11,"Let's talk... YOUTUBE-MONEY, ZOELLA & PEOPLE C...",Caspar,,2017-11-20T17:08:03.000Z,"[""caspar lee"", ""talk"", ""youtube"", ""zoella"", ""y...",816123,78829,1815,3692,https://i.ytimg.com/vi/PCVtE3wRNPg/default.jpg,False,False,False,I just wanted to talk about a few things on my...,GB,2017-11-20 17:08:03,0.0,2017.0,11.0,20.0,17.0,2017-11-23 23:59:59,3.0,2017.0,11.0,23.0,3.286065,0.023025,0.096590,0.002224,0.004524,59390.0,3023.0,336.0,151.0,12.0,1.034877e+06,88880.833333,3216.916667,4247.583333,248358.765269,11,let talk youtubemoney zoella people changing,53,7,33,0.622642,7,0.132075,7,0.132075,0,wanted talk thing mind moment n follow caspar ...,413,34,27,0.065375,292,0.707022,15,0.147700,"[""http://instagram.com/caspar"", ""http://twitte...",7,0,False,True,True,GB,141.471042,101.775891,96.967980,112.352044,110.953154,111.977977,"[36978, 2643, 1281, 348, 714, 457, 197, 139, 1...","[36978, 2643, 1281, 348, 714, 457, 197, 139, 1...","[36978, 2643, 1281, 348, 714, 457, 197, 139, 1...",4.997187,\n,NaN,talk lets,talk let,"[""[[344, 130], [448, 168]]"", ""[[225, 133], [33...","[338.5, 149.75]",338.5,149.75,0.0,"[0.17629368603229523, 1.7783278226852417, 1.37..."
1814,oEtEBOy0ys8,18.02.02,THINGS WE ALL DID IN BRITISH SECONDARY SCHOOL,Luke Birch,,2018-01-25T20:41:17.000Z,"[""British Secondary School"", ""Things we all di...",108685,9646,105,2013,https://i.ytimg.com/vi/oEtEBOy0ys8/default.jpg,False,False,False,A few relatable things that we all do in Briti...,GB,2018-01-25 20:41:17,3.0,2018.0,1.0,25.0,20.0,2018-02-02 23:59:59,4.0,2018.0,2.0,2.0,8.137986,0.010885,0.088752,0.000966,0.018521,9595.0,687.0,14.0,165.0,22.0,2.612937e+05,19651.363636,369.136364,4106.681818,13355.269782,14,thing british secondary school,45,8,38,0.844444,0,0.000000,0,0.000000,0,relatable thing british secondary schoolbritis...,1012,80,167,0.165020,526,0.519763,133,0.209486,"[""https://twitter.com/LukeBirch"", ""http://inst...",10,0,True,True,True,GB,99.504589,94.060093,94.125723,87.152131,76.271129,76.082382,"[38292, 2178, 974, 331, 812, 386, 400, 196, 19...","[38292, 2178, 974, 331, 812, 386, 400, 196, 19...","[38292, 2178, 974, 331, 812, 386, 400, 196, 19...",6.700530,\n,NaN,seatches,seatches,"[""[[202, 184], [260, 225]]""]","[231.0, 204.5]",231.0,204.50,0.0,"[0.43880921602249146, 2.079106569290161, 2.361..."
5545,P2I7hQHOqNI,18.18.01,Martin Luther King - A Time to Break Silence,FacelesswithEyesOpen,,2011-03-27T01:52:47.000Z,"[""Martin Luther King"", ""MLK"", ""A Time to Break...",2896,30,1,3,https://i.ytimg.com/vi/P2I7hQHOqNI/default.jpg,False,False,False,Martin Luther King - A Time to Break Silence S...,US,2011-03-27 01:52:47,6.0,2011.0,3.0,27.0,1.0,2018-01-18 23:59:5

In [11]:
df[~(df["category_id"] == '')].to_csv("./../../our_data/Etap3/GB_US-Etap3.csv", index=False)

In [2]:
dfGB = pd.read_csv(f"./../../our_data/Etap2/GB-Etap2.csv")
dfUS = pd.read_csv(f"./../../our_data/Etap2/US-Etap2.csv")
dfGB[:3]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,region,pub_date,pub_day_of_the_week,pub_year,pub_month,pub_day,pub_hour,tren_date,tren_day_of_the_week,tren_year,tren_month,tren_day,time_to_trend_in_days,dislikes/likes,likes/views,dislikes/views,comment_count/views,increase_views,increase_likes,increase_dislikes,increase_comms,times_in_trend,avg_views,avg_likes,avg_dislikes,avg_comms,avg_views_increase_per_hour,n_of_tags,title_clean,title_length,title_n_of_words,title_capital_letters,title_capital_letters_percent,title_small_letters,title_small_letters_percent,title_punctuation,title_punctuation_percent,title_n_of_emojis,desc_clean,desc_length,desc_n_of_words,desc_capital_letters,desc_capital_letters_percent,desc_small_letters,desc_small_letters_percent,desc_punctuation,desc_punctuation_percent,desc_list_of_urls,desc_n_of_urls,desc_n_of_emojis,isFacebook,isTwitter,isInstagram,region.1,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY,ocr_sentiment_compound,features_vec_resnet18
0,Jw1Y-zhQURU,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,NaN,2017-11-10T07:38:29.000Z,"[""christmas"", ""john lewis christmas"", ""john le...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...,GB,2017-11-10 07:38:29,4.0,2017.0,11.0,10.0,7.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,4.681597,0.184030,0.007707,0.001418,0.001312,338739.0,2528.0,612.0,-2359.0,14.0,8.229008e+06,61558.357143,11834.857143,7919.785714,1.543173e+06,9,john lewis christmas ad 2017 mozthemonster,45,7,7,0.155556,26,0.577778,2,0.044444,0,click continue story make monsternhttpbitly2mb...,821,98,39,0.047503,608,0.740560,9,0.085262,"[""http://bit.ly/2mboXgj"", ""http://bit.ly/2hg04...",7,0,True,True,True,GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],"[-1, -1]",-1.0,-1.000000,0.0,NaN
1,3s1rvMFUweQ,17.14.11,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,NaN,2017-11-12T06:24:44.000Z,"[""SNL"", ""Saturday Night Live"", ""SNL Season 43""...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs …Ready for...,GB,2017-11-12 06:24:44,6.0,2017.0,11.0,12.0,6.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,2.732813,0.089746,0.024260,0.002177,0.002617,615290.0,8211.0,783.0,715.0,3.0,1.499560e+06,31691.333333,2861.666667,3271.333333,3.855486e+05,36,taylor swift …ready live snl,41,8,8,0.195122,20,0.487805,5,0.121951,0,musical guest taylor swift performs …ready sat...,417,29,40,0.095923,276,0.661871,18,0.167866,"[""http://www.nbc.com/saturday"", ""http://www.nb...",7,0,True,True,True,GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],"[-1, -1]",-1.0,-1.000000,0.0,NaN
2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,NaN,2017-11-10T17:00:03.000Z,"[""Eminem"", ""Walk"", ""On"", ""Water"", ""Aftermath/S...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,GB,2017-11-10 17:00:03,4.0,2017.0,11.0,10.0,17.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,4.291620,0.055142,0.045891,0.002531,0.007336,3380838.0,53222.0,4295.0,-1646.0,14.0,3.017875e+07,950319.000000,56545.571429,130640.500000,3.998159e+06,6,eminem walk water audio ft beyoncé,42,8,6,0.142857,25,0.595238,4,0.095238,0,eminems new track walk water ft beyoncé availa...,594,34,29,0.048822,433,0.728956,9,0.158249,"[""http://shady.sr/WOWEminem"", ""https://goo.gl/...",14,0,True,True,True,GB,66.50283,71.743704,79.699635,83.992492,83.883918,83.748135,"[43570, 1539, 1228, 1489, 1782, 1839, 1918, 20...","[43570, 1539, 1228, 1489, 1782,

In [5]:
l = dfGB.loc[:, "video_id"].values

In [9]:
dfUS.loc[dfUS.video_id.isin(l), :]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,region,pub_date,pub_day_of_the_week,pub_year,pub_month,pub_day,pub_hour,tren_date,tren_day_of_the_week,tren_year,tren_month,tren_day,time_to_trend_in_days,dislikes/likes,likes/views,dislikes/views,comment_count/views,increase_views,increase_likes,increase_dislikes,increase_comms,times_in_trend,avg_views,avg_likes,avg_dislikes,avg_comms,avg_views_increase_per_hour,n_of_tags,title_clean,title_length,title_n_of_words,title_capital_letters,title_capital_letters_percent,title_small_letters,title_small_letters_percent,title_punctuation,title_punctuation_percent,title_n_of_emojis,desc_clean,desc_length,desc_n_of_words,desc_capital_letters,desc_capital_letters_percent,desc_small_letters,desc_small_letters_percent,desc_punctuation,desc_punctuation_percent,desc_list_of_urls,desc_n_of_urls,desc_n_of_emojis,isFacebook,isTwitter,isInstagram,region.1,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY,ocr_sentiment_compound,features_vec_resnet18
12,5E4ZBSInqUU,17.14.11,Marshmello - Blocks (Official Music Video),marshmello,NaN,2017-11-13T17:00:00.000Z,"[""marshmello"", ""blocks"", ""marshmello blocks"", ...",687582,114188,1333,8371,https://i.ytimg.com/vi/5E4ZBSInqUU/default.jpg,False,False,False,WATCH SILENCE MUSIC VIDEO ▶ https://youtu.be/T...,US,2017-11-13 17:00:00,0.0,2017.0,11.0,13.0,17.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,1.291655,0.011674,0.166072,0.001939,0.012175,1597554.0,66096.0,2066.0,3329.0,7.0,3.306325e+06,199726.857143,3785.571429,12575.428571,5.323263e+05,23,marshmello block official music video,42,6,5,0.119048,29,0.690476,3,0.071429,0,watch silence music video ▶ httpsyoutubetx1sqy...,1012,105,252,0.249012,539,0.532609,24,0.094862,"[""https://youtu.be/Tx1sqYc3qas"", ""https://yout...",14,1,True,True,True,US,110.327627,111.917650,109.217078,79.736369,79.131531,76.850568,"[39644, 2245, 786, 323, 108, 54, 27, 15, 8, 6,...","[39644, 2245, 786, 323, 108, 54, 27, 15, 8, 6,...","[39644, 2245, 786, 323, 108, 54, 27, 15, 8, 6,...",6.487640,\r\n,NaN,NaN,NaN,[],"[-1, -1]",-1.000000,-1.000000,0.0000,"[1.4723857641220093, 2.2314674854278564, 1.821..."
15,kgaO45SyaO4,17.14.11,The New SpotMini,BostonDynamics,28.0,2017-11-13T20:09:58.000Z,"[""Robots"", ""Boston Dynamics"", ""SpotMini"", ""Leg...",75752,9419,52,1230,https://i.ytimg.com/vi/kgaO45SyaO4/default.jpg,False,False,False,For more information . . . stay tuned.,US,2017-11-13 20:09:58,0.0,2017.0,11.0,13.0,20.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,1.159734,0.005521,0.124340,0.000686,0.016237,3626011.0,47142.0,2443.0,10378.0,3.0,3.103074e+06,46235.666667,1969.666667,9218.000000,6.531844e+04,5,new spotmini,16,3,4,0.250000,10,0.625000,0,0.000000,0,information stay tuned,38,8,1,0.026316,26,0.684211,4,0.105263,[],0,0,False,False,False,US,59.360469,72.502355,59.756227,59.882869,56.749708,52.073333,"[39801, 2305, 1554, 943, 827, 664, 537, 498, 4...","[39801, 2305, 1554, 943, 827, 664, 537, 498, 4...","[39801, 2305, 1554, 943, 827, 664, 537, 498, 4...",6.564880,\r\n,NaN,bostondyranks,bostondyranks,"[""[[211, 105], [263, 120]]""]","[237.0, 112.5]",237.000000,112.500000,0.0000,"[1.322217345237732, 0.8881644606590271, 0.2890..."
32,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,NaN,2017-11-10T17:00:03.000Z,"[""Eminem"", ""Walk"", ""On"", ""Water"", ""Aftermath/S...",17158531,787419,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,US,2017-11-10 17:00:03,4.0,2017.0,11.0,10.0,17.0,2017-11-14 23:59:59,1.0,2017.0,11.0,14.0,4.291620,0.055142,0.045891,0.002531,0.007336,3380886.0,53223.0,4295.0,-1646.0,5.0,2.228538e+07,860106.000000,49400.600000,125254.